In [6]:

!pip -q uninstall -y pymc arviz pytensor numpy
!pip -q install numpy==1.26.4 pymc==5.10.4 arviz==0.17.1 pytensor==2.18.6

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az

from google.colab import files
uploaded = files.upload()
csv_name = [k for k in uploaded.keys() if k.lower().endswith(".csv")][0]
df = pd.read_csv(csv_name)

# 1
cols = ["rentals","temp_c","humidity","wind_kph","is_holiday","season"]
df = df[cols].dropna().copy()

print(df.shape)
print(df.dtypes)
display(df.describe(include="all").T)

pd.plotting.scatter_matrix(df[["rentals","temp_c","humidity","wind_kph"]], figsize=(10,10), diagonal="hist")
plt.show()

for c in ["temp_c","humidity","wind_kph"]:
    plt.figure()
    plt.scatter(df[c], df["rentals"], s=10, alpha=0.6)
    plt.xlabel(c)
    plt.ylabel("rentals")
    plt.show()

# 2.a
y = df["rentals"].to_numpy(float)
X_cont = df[["temp_c","humidity","wind_kph"]].to_numpy(float)

x_cont_mean = X_cont.mean(axis=0)
x_cont_std = X_cont.std(axis=0, ddof=0)
X_cont_z = (X_cont - x_cont_mean) / x_cont_std

y_mean = y.mean()
y_std = y.std(ddof=0)
y_z = (y - y_mean) / y_std

is_holiday = df["is_holiday"].to_numpy(int)

season = pd.Categorical(df["season"])
season_dum = pd.get_dummies(season, drop_first=True)
season_cols = list(season_dum.columns)
X_season = season_dum.to_numpy(float)

X_lin = np.column_stack([X_cont_z, is_holiday, X_season])
lin_cols = ["temp_c","humidity","wind_kph","is_holiday", *season_cols]

temp_c_z = X_cont_z[:, 0]
temp_c2_z = temp_c_z**2
X_poly = np.column_stack([X_cont_z, temp_c2_z, is_holiday, X_season])
poly_cols = ["temp_c","humidity","wind_kph","temp_c2","is_holiday", *season_cols]

# 2.b
coords_lin = {"obs_id": np.arange(len(df)), "feature": lin_cols}
with pm.Model(coords=coords_lin) as m_lin:
    X = pm.Data("X", X_lin, dims=("obs_id","feature"))
    y_obs = pm.Data("y_obs", y_z, dims=("obs_id",))
    alpha = pm.Normal("alpha", 0, 1)
    beta = pm.Normal("beta", 0, 1, dims=("feature",))
    sigma = pm.HalfNormal("sigma", 1)
    mu = alpha + pm.math.dot(X, beta)
    pm.Normal("likelihood", mu=mu, sigma=sigma, observed=y_obs, dims=("obs_id",))
    idata_lin = pm.sample(2000, tune=2000, chains=4, target_accept=0.92, random_seed=42)
    idata_lin = pm.sample_posterior_predictive(idata_lin, extend_inferencedata=True, random_seed=42)

display(az.summary(idata_lin, var_names=["alpha","beta","sigma"], hdi_prob=0.95))

# 2.c
coords_poly = {"obs_id": np.arange(len(df)), "feature": poly_cols}
with pm.Model(coords=coords_poly) as m_poly:
    X = pm.Data("X", X_poly, dims=("obs_id","feature"))
    y_obs = pm.Data("y_obs", y_z, dims=("obs_id",))
    alpha = pm.Normal("alpha", 0, 1)
    beta = pm.Normal("beta", 0, 1, dims=("feature",))
    sigma = pm.HalfNormal("sigma", 1)
    mu = alpha + pm.math.dot(X, beta)
    pm.Normal("likelihood", mu=mu, sigma=sigma, observed=y_obs, dims=("obs_id",))
    idata_poly = pm.sample(2000, tune=2000, chains=4, target_accept=0.92, random_seed=42)
    idata_poly = pm.sample_posterior_predictive(idata_poly, extend_inferencedata=True, random_seed=42)

display(az.summary(idata_poly, var_names=["alpha","beta","sigma"], hdi_prob=0.95))

# 3
az.plot_trace(idata_lin, var_names=["alpha","beta","sigma"])
plt.show()
az.plot_trace(idata_poly, var_names=["alpha","beta","sigma"])
plt.show()

sum_lin = az.summary(idata_lin, var_names=["beta"], hdi_prob=0.95)
sum_poly = az.summary(idata_poly, var_names=["beta"], hdi_prob=0.95)
sum_lin["abs_mean"] = sum_lin["mean"].abs()
sum_poly["abs_mean"] = sum_poly["mean"].abs()
display(sum_lin.sort_values("abs_mean", ascending=False))
display(sum_poly.sort_values("abs_mean", ascending=False))

# 4.a
display(az.compare({"lin": idata_lin, "poly": idata_poly}, ic="loo"))
display(az.compare({"lin": idata_lin, "poly": idata_poly}, ic="waic"))

# 4.b
az.plot_ppc(idata_lin, group="posterior_predictive", num_pp_samples=200)
plt.show()
az.plot_ppc(idata_poly, group="posterior_predictive", num_pp_samples=200)
plt.show()

temp_grid = np.linspace(df["temp_c"].min(), df["temp_c"].max(), 80)
temp_grid_z = (temp_grid - x_cont_mean[0]) / x_cont_std[0]
hum0 = 0.0
wind0 = 0.0
is_h0 = 0
season0 = np.zeros((len(temp_grid), X_season.shape[1]))

Xg_lin = np.column_stack([temp_grid_z, np.full_like(temp_grid_z, hum0), np.full_like(temp_grid_z, wind0), np.full_like(temp_grid_z, is_h0), season0])
Xg_poly = np.column_stack([temp_grid_z, np.full_like(temp_grid_z, hum0), np.full_like(temp_grid_z, wind0), temp_grid_z**2, np.full_like(temp_grid_z, is_h0), season0])

with m_lin:
    pm.set_data({"X": Xg_lin})
    ppc_lin_grid = pm.sample_posterior_predictive(idata_lin, var_names=["likelihood"], random_seed=42, return_inferencedata=False)

with m_poly:
    pm.set_data({"X": Xg_poly})
    ppc_poly_grid = pm.sample_posterior_predictive(idata_poly, var_names=["likelihood"], random_seed=42, return_inferencedata=False)

def mean_hdi_orig(ppc_dict, y_mean, y_std, hdi_prob=0.95):
    yrep = np.asarray(ppc_dict["likelihood"])
    if yrep.ndim == 3:
        yrep = yrep.reshape(-1, yrep.shape[-1])
    mean_z = yrep.mean(axis=0)
    hdi_z = az.hdi(yrep, hdi_prob=hdi_prob)
    mean = mean_z * y_std + y_mean
    hdi = hdi_z * y_std + y_mean
    return mean, hdi

mean_lin, hdi_lin = mean_hdi_orig(ppc_lin_grid, y_mean, y_std, 0.95)
mean_poly, hdi_poly = mean_hdi_orig(ppc_poly_grid, y_mean, y_std, 0.95)

plt.figure()
plt.scatter(df["temp_c"], df["rentals"], s=10, alpha=0.4)
plt.plot(temp_grid, mean_lin)
plt.fill_between(temp_grid, hdi_lin[:,0], hdi_lin[:,1], alpha=0.2)
plt.xlabel("temp_c")
plt.ylabel("rentals")
plt.show()

plt.figure()
plt.scatter(df["temp_c"], df["rentals"], s=10, alpha=0.4)
plt.plot(temp_grid, mean_poly)
plt.fill_between(temp_grid, hdi_poly[:,0], hdi_poly[:,1], alpha=0.2)
plt.xlabel("temp_c")
plt.ylabel("rentals")
plt.show()

# 5
Q = np.quantile(df["rentals"].to_numpy(float), 0.75)
is_high_demand = (df["rentals"].to_numpy(float) >= Q).astype(int)
print(Q, is_high_demand.mean())

# 6
coords_logit_lin = {"obs_id": np.arange(len(df)), "feature": lin_cols}
with pm.Model(coords=coords_logit_lin) as m_logit_lin:
    X = pm.Data("X", X_lin, dims=("obs_id","feature"))
    y_bin = pm.Data("y_bin", is_high_demand, dims=("obs_id",))
    alpha = pm.Normal("alpha", 0, 1)
    beta = pm.Normal("beta", 0, 1, dims=("feature",))
    logit_p = alpha + pm.math.dot(X, beta)
    pm.Bernoulli("likelihood", logit_p=logit_p, observed=y_bin, dims=("obs_id",))
    idata_logit_lin = pm.sample(2000, tune=2000, chains=4, target_accept=0.92, random_seed=42)

display(az.summary(idata_logit_lin, var_names=["alpha","beta"], hdi_prob=0.95))

coords_logit_poly = {"obs_id": np.arange(len(df)), "feature": poly_cols}
with pm.Model(coords=coords_logit_poly) as m_logit_poly:
    X = pm.Data("X", X_poly, dims=("obs_id","feature"))
    y_bin = pm.Data("y_bin", is_high_demand, dims=("obs_id",))
    alpha = pm.Normal("alpha", 0, 1)
    beta = pm.Normal("beta", 0, 1, dims=("feature",))
    logit_p = alpha + pm.math.dot(X, beta)
    pm.Bernoulli("likelihood", logit_p=logit_p, observed=y_bin, dims=("obs_id",))
    idata_logit_poly = pm.sample(2000, tune=2000, chains=4, target_accept=0.92, random_seed=42)

display(az.summary(idata_logit_poly, var_names=["alpha","beta"], hdi_prob=0.95))

# 7
sum_logit_lin = az.summary(idata_logit_lin, var_names=["beta"], hdi_prob=0.95)
sum_logit_poly = az.summary(idata_logit_poly, var_names=["beta"], hdi_prob=0.95)
sum_logit_lin["abs_mean"] = sum_logit_lin["mean"].abs()
sum_logit_poly["abs_mean"] = sum_logit_poly["mean"].abs()
display(sum_logit_lin.sort_values("abs_mean", ascending=False))
display(sum_logit_poly.sort_values("abs_mean", ascending=False))

display(az.compare({"logit_lin": idata_logit_lin, "logit_poly": idata_logit_poly}, ic="loo"))
display(az.compare({"logit_lin": idata_logit_lin, "logit_poly": idata_logit_poly}, ic="waic"))

az.plot_forest(idata_logit_lin, var_names=["beta"], combined=True, hdi_prob=0.95)
plt.show()
az.plot_forest(idata_logit_poly, var_names=["beta"], combined=True, hdi_prob=0.95)
plt.show()




ERROR: Ignored the following yanked versions: 2.33.0
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 2.17.2 Requires-Python <3.12,>=3.9; 2.17.3 Requires-Python <3.12,>=3.9; 2.17.4 Requires-Python <3.12,>=3.9; 2.18.0 Requires-Python <3.12,>=3.9; 2.18.1 Requires-Python <3.12,>=3.9; 2.18.2 Requires-Python <3.12,>=3.9; 2.18.3 Requires-Python <3.12,>=3.9; 2.18.4 Requires-Python <3.12,>=3.9; 2.18.5 Requires-Python <3.12,>=3.9; 2.18.6 Requires-Python <3.12,>=3.9
ERROR: Could not find a version that satisfies the requirement pytensor==2.18.6 (from versions: 2.8.10, 2.8.11, 2.9.1, 2.10.0, 2.10.1, 2.11.0, 2.11.1, 2.11.2, 2.12.0, 2.12.1, 2.12.2, 2.12.3, 2.13.0, 2.13.1, 2.14.0, 2.14.1, 2.14.2, 2.15.0, 2.16.0, 2.16.1, 2.16.2, 2.16.3, 2.17.0, 2.17.1, 2.19.0, 2.20.0, 2.21.0, 2.22.0, 2.22.1, 2.

ModuleNotFoundError: No module named 'pymc'

1.
temp_c vs rentals: relația este neliniară. Numărul de închirieri crește odată cu temperatura până la un anumit punct, după care creșterea încetinește și tinde să se plafoneze.

humidity vs rentals: există o relație negativă aproximativ liniară. Pe măsură ce umiditatea crește, numărul de închirieri tinde să scadă, cu o dispersie relativ constantă.

wind_kph vs rentals: relația este slab negativă, aproape liniară, dar cu variabilitate mare; viteza vântului pare să aibă un efect mai mic comparativ cu temperatura sau umiditatea.

Se observă ca pentru temperaturi moderate–ridicate, variabilitatea lui rentals este mai mare decât pentru temperaturi scăzute.

4 a Modelul polinomial are scoruri mai bune decât modelul liniar.


4 b. modelul polinomial este mai adecvat din punct de vedere predictiv.

7. temperatura este factorul care influențează cel mai mult probabilitatea unei zile de high demand